## AI Agent智能应用从0到1定制开发 
## AI Agent Intelligent Application Custom Development from 0 to 1
******
- 此代码为网课《AI Agent智能应用从0到1定制开发》的配套代码，需要注意本套代码建议与网课适配配合食用。
- This code for the online course <AI Agent Intelligent Applications from 0 to 1 custom development> supporting code, need to pay attention to this set of code is recommended with the online course adapted to work with consumption.
- 需要注意由于课程开发周期的原因，langchain版本跨越了3个大版本，部分代码会与视频演示有差别!
- Note that due to the course development cycle, the langchain version spans 3 major releases and some of the code will differ from the video demo!
- 课程地址：https://coding.imooc.com/class/822.html
- Course address: https://coding.imooc.com/class/822.html

### 从环境变量中读取密钥
### Read the key from the environment variable
- 注意：尽量将你的OpenAI Key存储在类似.env文件中，而不是明文暴露在代码里，这是一种基本的安全措施
- Note: Try to store your OpenAI Key in something like an .env file, rather than exposing it explicitly in code, as a basic safety measure!
******

In [1]:

import os
from dotenv import load_dotenv
# Load environment variables from openai.env file
load_dotenv("asset/openai.env")

# Read the OPENAI_API_KEY from the environment
api_key = os.getenv("OPENAI_API_KEY")
api_base = os.getenv("OPENAI_API_BASE")
os.environ["OPENAI_API_KEY"] = api_key
os.environ["OPENAI_API_BASE"] = api_base
os.environ["SERPAPI_API_KEY"] = os.getenv("SERPAPI_API_KEY")

### 第一个agent
### The first agent
- 会做数学题
- Can do math.
- 不知道答案的时候可以搜索
- If you don't know the answer, you can search for it.
<hr>

In [2]:
#定义llm
#defining llm
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    temperature=0,
    model="gpt-3.5-turbo",
)

-搭建工具
- serpai是一个聚合搜索引擎，需要安装谷歌搜索包以及申请账号 https://serpapi.com/manage-api-key
- llm-math是一个封装好的数学计算链
- Building tools
- serpai is an aggregation search engine that requires the installation of the Google search package and the application for an account at https://serpapi.com/manage-api-key
- llm-math is a packaged mathematical computation chain

In [3]:
! pip install google-search-results

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for google-search-results: filename=google_search_results-2.4.2-py3-none-any.whl size=32083 sha256=0a4ad851aa394db97144b92108d7d2043d80362f9a7ff4b55f0f8549d67850b4
  Stored in directory: c:\users\administrator\appdata\local\pip\cache\wheels\0c\47\f5\89b7e770ab2996baf8c910e7353d6391e373075a0ac213519e
Successfully built google-search-results



[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
! pip install numexpr

  Obtaining dependency information for numexpr from https://files.pythonhosted.org/packages/f5/4f/6c2621db459b8cab5cfcbd21c6ccc6cde594d0256853d9a79e4f454ed131/numexpr-2.10.1-cp312-cp312-win_amd64.whl.metadata
   ---------------------------------------- 0.0/141.3 kB ? eta -:--:--
   -- ------------------------------------- 10.2/141.3 kB ? eta -:--:--
   -------- ------------------------------ 30.7/141.3 kB 330.3 kB/s eta 0:00:01
   ------------------- ------------------- 71.7/141.3 kB 491.5 kB/s eta 0:00:01
   -------------------------------------- 141.3/141.3 kB 839.8 kB/s eta 0:00:00



[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
from langchain.agents import load_tools

tools = load_tools(["serpapi","llm-math"], llm=llm)

- 定义agent,使用小样本增强生成类型
- Define agent, use small samples to enhance generation type

In [7]:
from langchain.agents import initialize_agent
from langchain.agents import AgentType

agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,#这里有不同的类型
    verbose=True,#是否打印日志
)

e:\教程\imooc\langchan_course\langchain_v0.1\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


In [8]:
agent.invoke("请问现任的美国总统是谁？他的年龄的平方是多少?")



> Entering new AgentExecutor chain...
I need to find out who the current President of the United States is and then calculate the square of his age.
Action: Search
Action Input: Current President of the United States
Observation: Joe Biden
Thought:Now I need to find out Joe Biden's age and calculate the square of it.
Action: Search
Action Input: Joe Biden age
Observation: 81 years
Thought:I will now calculate the square of Joe Biden's age, which is 81.
Action: Calculator
Action Input: 81^2
Observation: Answer: 6561
Thought:I now know the final answer.
Final Answer: 6561

> Finished chain.


{'input': '请问现任的美国总统是谁？他的年龄的平方是多少?', 'output': '6561'}

- 注意：上方的告警，initialize_agent是一个老的API，在0.2.0中将被启用，推荐使用类似  create_react_agent, create_json_agent, create_structured_chat_agent这样的API来构建
- "Note: The warning above indicates that initialize_agent is an old API that will be enabled in 0.2.0. It is recommended to use APIs like create_react_agent, create_json_agent, and create_structured_chat_agent to build."